## **Arrow Detection with Direction**

Import Libraries

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from keras import Sequential
from keras.layers import Dense
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten, Conv2DTranspose

import cv2 
import numpy as np 
from keras.datasets import cifar10 
from keras import backend as K 
from keras.utils import np_utils

import math 
from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler

print(tf.__version__)

Generate & Save Dataset

In [0]:
path = '/content/drive/My Drive/Drive1(Own)/Arrow224/train/'
classes = os.listdir(path)
train_size = 704
file_size = 160
print(classes)
index = 0
counter = 0
X_train = np.zeros((train_size, int(file_size), int(file_size), 3), dtype = 'uint8')
Y_train = np.zeros((train_size), dtype = 'uint8')

for i in classes:

    print('class: ', i)
    files = os.listdir(str(path) + str(i))

    for k in files:
        img = Image.open(str(path) + str(i) + '/' + str(k))
        img.load
        img = img.resize((int(file_size), int(file_size)), Image.ANTIALIAS)
        npimg = np.asarray( img, dtype="uint8" )
        X_train[counter,:,:,:] = npimg
        Y_train[counter] = classes.index(i)
        counter += 1
    
print(counter)

In [0]:
train_size = 704
#test_size = 70

train_images = np.resize(X_train, (train_size, 160, 160, 3)) / 255
train_labels = np.resize(Y_train, (train_size))

#test_images = np.resize(X_train, (test_size, 160, 160, 3)) / 255
#test_labels = np.resize(Y_test, (test_size))

print(train_images.shape, train_labels.shape)
#print(test_images.shape, test_labels.shape)


Define Network

In [0]:
in_size = 16
x = keras.layers.Input(shape=(160, 160, 3))

gen1 = keras.layers.Conv2D(8, (4, 4), padding = 'SAME', activation = 'relu')(x)
P1 = keras.layers.MaxPooling2D(2, padding = 'SAME')(gen1)

gen2 = keras.layers.Conv2D(16, (4, 4), padding = 'SAME', activation = 'relu')(P1)
P2 = keras.layers.MaxPooling2D(4, padding = 'SAME')(gen2)

gen3 = keras.layers.Conv2D(32, (4, 4), padding = 'SAME', activation = 'relu')(P2)
P3 = keras.layers.MaxPooling2D(2, padding = 'SAME')(gen3)

gen4 = keras.layers.Conv2D(16, (4, 4), padding = 'SAME', activation = 'relu')(P3)
P4 = keras.layers.MaxPooling2D(4, padding = 'SAME')(gen4)

gen4 = keras.layers.Conv2D(8, (4, 4), padding = 'SAME', activation = 'sigmoid')(P4)
P4 = keras.layers.MaxPooling2D(2, padding = 'SAME')(gen4)

x4 = keras.layers.Flatten()(P4)
x5 = keras.layers.Dense(3, activation = 'softmax')(x4)

model = keras.models.Model([x], x5)
model.summary()

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Training

In [0]:
model.fit(train_images, train_labels, epochs=15, batch_size = 8)

Save Weights

In [0]:
model.save_weights('/content/sui_arrow.h5')

Test random images

In [0]:
img = Image.open('/content/drive/My Drive/Drive1(Own)/Arrow224/test/right/Untitled16.jpg')
img.load
plt.imshow(img)
plt.show()
img = img.resize((160, 160), Image.ANTIALIAS)
npimg = np.asarray(img, dtype="uint8" )
print('shape', np.shape(npimg))
test_image = np.resize(npimg, (1, 160, 160, 3))
print('shape', np.shape(test_image))
predictions = model.predict(test_image)
print(predictions)
classes = ['L', 'R', 'N']
print(classes[np.argmax(predictions)])